# <center>Gathering Supply Chain Data<center>

- Date: March 2019
- Author: Minsu Yeom, CFA, FRM

In [1]:
import os
import numpy as np
import pandas as pd
import bamboolib as bam
import xlwings as xw
import datetime as dt
import xlrd
from tqdm import tqdm
from datetime import date
from utils.misc import get_excel_column_name, get_last_bday
from gathering.EquityUniverse import *

In [2]:
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload
%autoreload 2

# 1. Load Equity Universe

In [3]:
universe_loader = EquityUniverse('./dataset/universe/eqy_universe_gt50mm.log')
universe = universe_loader.get_universe()

### Missing value analysis

- Looks all good.

In [4]:
universe.isna().sum()

Identifier                     0
Name                           0
Revenue                        0
Company Type                   0
Business Description           0
FactSet Industry               0
Crunchbase Category(BETA)      0
Crunchbase Rank(BETA)          0
Ultimate Parent Name           0
Fiscal Year End                1
Country                        0
Website                      127
dtype: int64

# 2. Launch Excel
- `fdswFixExcel.exe` enables the add-ins.
- `retcode`: 0 if runs fine. 1 if any error.

In [5]:
import subprocess
retcode = subprocess.run(['C:/Program Files (x86)/FactSet/fdswFixExcel.exe'])

# 3. Set file names to save results

In [6]:
save_filename = './dataset/biz_segment/biz_segment'
meta_filename = './dataset/biz_segment/biz_segment_meta'

# 4. Set variables for a format in Excel

In [7]:
header_direction = 'H'  # 'H': a horizontal header, 'V': a vertical header
time_series = True  # True if data is time-series. False if cross-sectional.

# The number of rows for one complete table at a time.
# If it's time-series, it might be the maximum number of business segments in a year.
per_table_rows = 15

# The number of columns for one complete table at a time.
# If it's cross-sectional, it might be the maximum number of suppliers at a point of time.
per_table_cols = 11

# How many times we repeat on one worksheet.
# The height of the entire data would be per_table_rows*repeat_no
repeat_no = 10

#### Set an Excel header

In [8]:
header_meta = ['asof_fy', 'rpt_date', 'ticker']
header_fcst = [
    'biz_segment', 'sales', 'op_income', 'assets', 'capex', 'dep', 'restated',
    'frequency'
]

header = header_meta + header_fcst

# 6. Fill an Excel workbook

## Case 1
- `header_direction` == 'H' : The header in Excel is horizontal.
- `time_series` == True

### Define a function named `get_biz_segments()` that `get_factset_data()` points to.
- `get_biz_segments` is an arbitrary name, but `tickers` should be the only argument in that function.

In [9]:
# We drop any row containing an empty cell in `biz_segment_ column.
drop_rows_by = ('biz_segment', 'empty')

In [10]:
def get_biz_segments(tickers, restated = 0):
    """Returns business segments data.
    
    Arguments:
    ----------
    tickers: list
        A list of tickers for which the segements are extracted.
    
    restated: boolean
        If 0, the original, as-reported data will be extracted.
        If 1, the restated data will be extracted.
    
    Returns:
    --------
    ws: dictionary of Excel worksheet. Keys are ticker names.
        A list of worksheets, each of which contains data for one ticker.
    """
    ws = {}
    
    restated_param = 'ANN_R' if restated else 'ANN'
    
    # Iterate over worksheets
    for ticker in tickers:
        ws[ticker] = xw.sheets.add(ticker)
        
        ### We fill columns not required to do repeatedly: C, J and K.
        ## Column C: ticker
#         For rows: all (all tables)
#         fcst_col = ['C', 'J', 'K']
#         fcst_value = [ticker, restated_value, 'A']
        
#         for value_idx, col_nm in enumerate(fcst_col):
        fcst_range = '{:s}{:d}:{:s}{:d}'.format('C', 1, 'C', per_table_rows*repeat_no)
#         fcst_range = '{:s}{:d}'.format('C', 1)
        ws[ticker].range(fcst_range).value = ticker
        
        ### We fill the rest columns repeatedly.
        for j in range(repeat_no):
            ## Column A: as of fiscal year (asof_fy) ##
            # For rows: 1-15 (per table)
            # =FDS("GOOGL-US","JULIAN(FF_EPS_RPT_DATE(ANN_R,0AY,0AY,,R1).dates)")
            fds_fn = '","JULIAN(FF_EPS_RPT_DATE(' + restated_param + ','
            fds_param = ',,R1).dates)")'
            
            fcst_start_col, fcst_end_col = 'A', 'A'
            fcst_range = '{:s}{:d}:{:s}{:d}'.format(fcst_start_col, j*per_table_rows+1, fcst_end_col, (j+1)*per_table_rows)
#             fcst_range = '{:s}{:d}'.format(fcst_start_col, j*per_table_rows+1)
            ws[ticker].range(fcst_range).options(dates=dt.datetime).value = '=FDS("' + ticker + fds_fn + str(-1*j) + 'AY,' + str(-1*j) + 'AY' + fds_param
            
            ## Column B: report date (rpt_date) ##
            # For rows: 1-15
            # =FDS("GOOGL-US","FF_EPS_RPT_DATE(ANN_R,0AY,0AY,,R1)")
            fds_fn = '","FF_EPS_RPT_DATE(' + restated_param + ','
            fds_param = ',,R1)")'
            
            fcst_start_col, fcst_end_col = 'B', 'B'
            fcst_range = '{:s}{:d}:{:s}{:d}'.format(fcst_start_col, j*per_table_rows+1, fcst_end_col, (j+1)*per_table_rows)
#             fcst_range = '{:s}{:d}'.format(fcst_start_col, j*per_table_rows+1)
            ws[ticker].range(fcst_range).value = '=FDS("' + ticker + fds_fn + str(-1*j) + 'AY,' + str(-1*j) + 'AY' + fds_param
            
            ## Column D: business segment names ##
            # For rows: 1-15 
            # =FDSB("GOOGL-US","FF_SEGMENT_RPT_LABELS(ANN,0,,,,,BUS,""ALL_TOT"",""ALL"")[1:15]")
            fds_fn = '", "FF_SEGMENT_RPT_LABELS(' + restated_param + ','
            fds_param = ',,,,,BUS,""ALL_TOT"",""ALL"")[1:15]")'
            
            fcst_start_col = get_excel_column_name(len(header_meta))
            fcst_end_col = fcst_start_col
            fcst_range = '{:s}{:d}:{:s}{:d}'.format(fcst_start_col, j*per_table_rows+1, fcst_end_col, (j+1)*per_table_rows-1)
            ws[ticker].range(fcst_range).formula_array = '=FDSB("' + ticker + fds_fn + str(-1*j) + fds_param
            
#             # For row: 16
#             fcst_range = '{:s}{:d}'.format(fcst_start_col, (j+1)*per_table_rows)
#             ws[ticker].range(fcst_range).value = 'Total'
            

            ## Column E-I: Sales, Operating Income, Assets, Capex, Depreciation ##
            # For rows: 1-15
            # =FDSB("GOOGL-US","FF_SEGMENT_RPT_DATA(ANN,0,,,,""SALES"",USD,BUS,""SEG"",M)[1:15]")
            biz_segment_col = fcst_start_col
            fds_fn = '", "FF_SEGMENT_RPT_DATA(' + restated_param + ','
            fds_param1 = ',,,""'
            fds_param2 = '"",USD,BUS,""ALL_TOT"",M)[1:15]")'
            
#             tot_fds_fn = '", "LAST_ITEM_AV(FF_SEGMENT_RPT_DATA(' + restated_param + ','
#             tot_fds_param1 = ',,,,""'
#             tot_fds_param2 = '"",USD,BUS,""SEG_TOT"",M)[1:15])")'
            
            for k, param in enumerate(['SALES', 'OPINC', 'ASSETS', 'CAPEX', 'DEP']):
                fcst_start_col = get_excel_column_name(k+1, letter=biz_segment_col)
                fcst_end_col = fcst_start_col
                fcst_range = '{:s}{:d}:{:s}{:d}'.format(fcst_start_col, j*per_table_rows+1, fcst_end_col, (j+1)*per_table_rows-1)
                ws[ticker].range(fcst_range).formula_array = '=FDSB("' + ticker + fds_fn + str(-1*j) + 'AY,'+ str(-1*j) + 'AY' + fds_param1 + param + fds_param2
                
#                 # For row: D6
#                 # =FDSB("GOOGL-US","LAST_ITEM_AV(FF_SEGMENT_RPT_DATA(ANN,0,,,,""SALES"",USD,BUS,""SEG_TOT"",M)[1:15])")
#                 fcst_range = '{:s}{:d}'.format(fcst_start_col, (j+1)*per_table_rows)
#                 ws[ticker].range(fcst_range).value = '=FDSB("' + ticker + tot_fds_fn + str(-1*j) + tot_fds_param1 + param + tot_fds_param2

    return ws

In [11]:
if header_direction == 'H' and time_series == True:
    # Excel column names for meata data in `header`
    meta_col = {}
    for i, col_nm in enumerate(header_meta):
        meta_col[col_nm] = get_excel_column_name(i) 
    
    # Set the whole data range on a worksheet.
    data_height = per_table_rows*repeat_no
    data_range = 'A1:' + get_excel_column_name(len(header)-1) + str(data_height)
    
    get_factset_data = get_biz_segments

In [12]:
df = pd.DataFrame()
ws_list = []
# total_sz = 12 # The total number of target companies to extract
total_sz = universe.shape[0] # The total number of target companies to extract
save_unit = 500
batch_sz = 1 # The number of target companies to be filled in Excel before it's deleted.

restated = 0

for idx in tqdm(range(0, total_sz, batch_sz)):
    tickers = universe.loc[idx:idx+batch_sz-1, 'Identifier'].to_list()
    ws = get_factset_data(tickers, restated)

    for sheet in ws.keys():
        ws_list.append(pd.DataFrame(ws[sheet].range(data_range).value))
        ws[sheet].delete()
    
    #Pickle each intermediate result per `save_unit` company.
    if idx != 0 and (idx % save_unit == 0):
        new_df = pd.concat(ws_list)
        
        # We drop any row containing {empty|na} in the column "drop_rows_by[0]"
        if drop_rows_by[1] == 'empty':
            new_df = new_df.loc[new_df.loc[:, header.index(drop_rows_by[0])].astype(bool), :]
        elif drop_rows_by[1] == 'na':
            new_df = new_df.dropna(subset=[header.index(drop_rows_by[0])])
            
        
        
        # Fill `restated` and `frequency` column with {0!1} and 'A', respectively.
        # We fill them here to avoid calling xlwings as little as possible.
        # 'A' indicates 'Annual' frequency
        new_df.loc[:, header.index('restated')] = restated
        new_df.loc[:, header.index('frequency')] = 'A'
            
        new_df.to_pickle(save_filename + '_' + str(idx) + '.pkl')
        df = pd.concat([df, new_df])        
        del ws_list
        del new_df
        ws_list = []

df.columns = header
df = df.dropna(subset=['asof_fy'])
df = df.reset_index(drop = True)

100%|████████████████████████████████████████████████████████████████████████| 14241/14241 [209:55:44<00:00, 53.07s/it]


In [32]:
# Changing numerical dates in Excel to pd.datetime.
df.loc[:, 'asof_fy'] = df.loc[:, 'asof_fy'].apply(xlrd.xldate_as_datetime, args=(0,))
df.loc[:, 'rpt_date'] = pd.to_datetime(df.loc[:, 'rpt_date'])

# Split business segment name, SIC and NAICS classification from `biz_segment` column.
biz_segment_split = df.loc[:, 'biz_segment'].str.split("|", expand=True)
biz_segment_split = biz_segment_split.replace(to_replace=[r'([A-Z])\w+=', np.nan], value='', regex=True)
df.loc[:, 'biz_segment'] = biz_segment_split.loc[:, 0]
df.loc[:, 'SIC'] = biz_segment_split.loc[:, 1]
df.loc[:, 'NAICS'] = biz_segment_split.loc[:, 2]

# Change column orders to make more sense.
old_columns = [x for x in df.columns if x not in ['SIC', 'NAICS']]
insert_index = old_columns.index('biz_segment') + 1
df = df[old_columns[:insert_index] + ['SIC', 'NAICS'] + old_columns[insert_index:]]

# change column types.
for col_nm in ['sales', 'op_income', 'assets', 'capex', 'dep', 'restated']:
    df.loc[:, col_nm] = pd.to_numeric(df.loc[:, col_nm], errors='coerce')

In [33]:
df.to_pickle(save_filename + '.pkl')
df.to_feather(save_filename + '.feather')

In [64]:
df

,asof_fy,rpt_date,ticker,biz_segment,SIC,NAICS,sales,op_income,assets,capex,dep,restated,frequency
0,2019-12-31,2020-03-18,2GO-PH,Non-shipping,4731,488510,242.38339,0.874630,123.880580,5.525330,3.859816,0,A
1,2019-12-31,2020-03-18,2GO-PH,Shipping,4482,483212,175.02292,-22.599566,217.678940,23.633623,35.578880,0,A
2,2019-12-31,2020-03-18,2GO-PH,Eliminations/Adjustments,9999,99999,NaN,-1.815924,-55.114693,NaN,NaN,0,A
3,2019-12-31,2020-03-18,2GO-PH,Consolidated Total,,,417.40631,-23.540859,286.444827,29.158953,39.438696,0,A
4,2018-12-31,2019-03-20,2GO-PH,Non-shipping,4731,488510,259.63773,-4.105494,162.048890,5.592721,3.635115,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
674820,2011-12-30,2012-02-06,UDR-US,Non-mature Communities/Other,6798,531110,215.45900,140.311000,3340.115000,9.500000,NaN,0,A
674821,2011-12-30,2012-02-06,UDR-US,Consolidated Total,,,730.95800,484.760000,8074.471000,57.800000,0.000000,0,A
674822,2010-12-31,2011-02-07,UDR-US,Same-store Communities,6798,531110,539.78800,357.960000,5022.281000,44.700000,NaN,0,A
674823,2010-12-31,2011-02-07,UDR-US,Non-mature Communities/Other,6798,531110,94.08000,53.104000,1859.066000,3.900000,NaN,0,A


## Case 2: a horizontal workbook

In [10]:
if header_direction == 'V':
    # Excel row numbers for meta data
    meta_row = {}
    for i, row_no in enumerate(header_meta):
        meta_row[row_no] = i+1

    per_company_cols = 500
    per_company_rows = len(header)
    # The number of companies to extract at one run. We do this number of times until we extract all information.
    batch_sz = 10   
    alphabet_count = 26

    fcst_start_col = 'A'
    fcst_end_col = get_excel_column_name(per_company_cols-1)
    fcst_start_row =  len(header_meta)+1
    fcst_end_row = per_company_rows
    
    # The whole data range
    data_height = len(header) * batch_sz
    data_range = [
        'A' + str(i * per_company_rows + 1) + ':' +
        str(get_excel_column_name(per_company_cols - 1)) + str(
            (i + 1) * per_company_rows) for i in range(batch_sz)
    ]

#### FactSet settings

In [12]:
# SUPL: Suppliers. 
# CUST: Customers
# PRTR: Partners
# COMP: Competitors
# It's FactSet's codes.
relationships = ['SUPL', 'CUST', 'PRTR', 'COMP']

# =FDS("GOOGL-US", "TRANSPOSE(FF_COMPANY_RELATIONSHIP(SUPL,PUB,Ticker,ALL,ALL))")
fds_fn = '", "TRANSPOSE(FF_COMPANY_RELATIONSHIP('
fds_param = ',PUB,Ticker,ALL,ALL))")'

#### Load company relationship data from FactSet and write it in an Excel workbook by
- Writing `=FDS()` directly an Excel sheet named as in `relationships`
- Allocating `rows` rows for each company

In [13]:
def get_company_relationships(tickers):
    ws = {}
    fcst_ranges = {}

    # Iterate over ['SUPL', 'CUST', 'PRTR', 'COMP']
    for i, rel in enumerate(relationships):
        ws[rel] = xw.sheets.add(rel)    # Create a sheet per relationship

        # Load company relationship data and write it in the workbook.
        for j, ticker in enumerate(tickers):
            fcst_ranges[ticker] = '{:s}{:d}:{:s}{:d}'.format(fcst_start_col, j*per_company_rows+fcst_start_row, fcst_end_col, j*per_company_rows+fcst_end_row)
            ws[rel].range(fcst_ranges[ticker]).formula_array = '=FDS("' + ticker + fds_fn + rel + fds_param
                        
            # For each company, we write Date, Ticker and Relationship for the corresponding rows in the workbook.
            for meta_type, row_no in meta_row.items():
                row = j*per_company_rows+row_no
                meta_range = '{:s}{:d}:{:s}{:d}'.format('A', row, fcst_end_col, row)
                meta_value = get_last_bday() if meta_type == 'date' else rel if meta_type == 'relationship' else ticker
                ws[rel].range(meta_range).value = meta_value
    
    return ws

#### Extract data from each sheet in `ws` and merge it into `df`
- We also do: resetting the index and setting the column names.

In [14]:
universe

,Identifier,Name,Revenue,Company Type,Business Description,FactSet Industry,Crunchbase Category(BETA),Crunchbase Rank(BETA),Ultimate Parent Name,Fiscal Year End,Country,Website
0,2GO-PH,"2GO Group, Inc.",413.47714,Public Company,"2GO Group, Inc. engages in the provision of pr...",Marine Shipping,-,-,"2GO Group, Inc.",2019-12-31,Philippines,http://www.2go.com.ph
1,601360-CN,"360 Security Technology, Inc.",1948.56120,Public Company,"360 Security Technology, Inc. provides interne...",Packaged Software,-,-,"360 Security Technology, Inc.",2018-12-31,China,http://www.360.cn
2,1361-HK,361 Degrees International Ltd.,815.04840,Public Company,361 Degrees International Ltd. is an investmen...,Apparel/Footwear,-,-,361 Degrees International Ltd.,2019-12-31,China,http://www.361sport.com
3,523395-IN,3M India Limited,431.16843,Public Company,3M India Ltd. engages in the business of indus...,Specialty Telecommunications,-,-,3M Company,2019-03-31,India,http://www.3mindia.in
4,1530-HK,"3SBio, Inc.",692.74005,Public Company,"3SBio, Inc. operates as in investment holding ...",Biotechnology,-,-,"3SBio, Inc.",2018-12-31,China,http://www.3sbio.com
...,...,...,...,...,...,...,...,...,...,...,...,...
14236,ZM-US,"Zoom Video Communications, Inc.",622.65800,Public Company,"Zoom Video Communications, Inc. engages in the...",Packaged Software,-,-,"Zoom Video Communications, Inc.",2020-01-31,United States,http://www.zoom.us
14237,ZS-US,"Zscaler, Inc.",302.83600,Public Company,"Zscaler, Inc. engages in the provision of clou...",Packaged Software,-,-,"Zscaler, Inc.",2019-07-31,United States,http://www.zscaler.com
14238,ZUMZ-US,Zumiez Inc.,1034.12900,Public Company,"Zumiez, Inc. engages in retailing apparel, foo...",Apparel/Footwear Retail,-,-,Zumiez Inc.,2020-02-01,United States,http://www.zumiez.com
14239,ZUO-US,"Zuora, Inc.",276.05700,Public Company,"Zuora, Inc. operates as an online subscription...",Packaged Software,-,-,"Zuora, Inc.",2020-01-31,United States,http://www.zuora.com


In [15]:
df = pd.DataFrame()
ws_list = []
# total_sz = 100 # The total number of target companies to extract
total_sz = universe.shape[0] # The total number of target companies to extract
save_unit = 500
for idx in tqdm(range(0, total_sz, batch_sz)):
#     print(idx, end = ' ')
    tickers = universe.loc[idx:idx+batch_sz-1, 'Identifier'].to_list()
    ws = get_company_relationships(tickers)

    for sheet in ws.keys():
        for r in data_range:
            ws_list.append(pd.DataFrame(ws[sheet].range(r).options(transpose=True).value))
        ws[sheet].delete()
    
    # Pickle each intermediate result per `save_unit` company.
    if idx != 0 and (idx % save_unit == 0):
        new_df = pd.concat(ws_list)
        new_df = new_df.dropna(subset=[header.index('rel_comp_tic')])
        new_df.to_pickle('./dataset/comprel/pickle/company_relationships_' + str(idx) + '.pkl')
        df = pd.concat([df, new_df])        
        del ws_list
        del new_df
        ws_list = []
    
df.columns = header
df = df.reset_index(drop = True)

100%|████████████████████████████████████████████████████████████████████████████| 1425/1425 [4:56:38<00:00, 12.49s/it]


In [16]:
df.to_pickle('./dataset/comprel/company_relationships.pkl')

In [17]:
df.to_feather('./dataset/comprel/company_relationships.feather')

## ----------------- The end of work -------------------